<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/pythorch_sku_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install torch


In [21]:
import pandas as pd
df = pd.read_excel('/content/mat_y_40_sku.xlsx')

In [22]:
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(Day=1)).dt.to_period('M')
df

,Month,Year,Sku_name,Analytic1,Articul8,Brand,PRICE_PER_UNIT,SALES_VOL,SHARE_VOL_PROMO,SHARE_VOL,Date
0,1,2022,"Сыр полутвёрдый BELFOR Флоу-пак 240 г 45,0%",240 г,Другой3,BELFOR,757.015534,68.88,0.971298,0.287575,2022-01
1,2,2022,"Сыр полутвёрдый BELFOR Флоу-пак 240 г 45,0%",240 г,Другой3,BELFOR,767.246528,2.88,0.491319,0.015748,2022-02
2,3,2022,"Сыр полутвёрдый BELFOR Флоу-пак 240 г 45,0%",240 г,Другой3,BELFOR,999.458333,0.24,0.000000,0.002016,2022-03
3,1,2024,Сыр полутвёрдый Casa Margot Эмменталер Флоу-па...,150 г,Эмменталер,Casa Margot,3024.421769,14.70,0.777143,1.000000,2024-01
4,2,2024,Сыр полутвёрдый Casa Margot Эмменталер Флоу-па...,150 г,Эмменталер,Casa Margot,3118.870635,12.60,0.726190,1.000000,2024-02
...,...,...,...,...,...,...,...,...,...,...,...
2030,10,2023,Сыр полутвёрдый Эконива Щучанский Флоу-пак 200...,200 г,Классический,Эконива,685.842388,14049.80,0.712729,0.027145,2023-10
2031,11,2022,Сыр полутвёрдый Эконива Щучанский Флоу-пак 200...,200 г,Классический,Эконива,851.198597,3244.00,0.410310,0.006087,2022-11
2032,11,2023,Сыр полутвёрдый Эконива Щучанский Флоу-пак 200...,200 г,Классический,Эконива,702.743684,12592.60,0.418561,0.026897,2023-11
2033,12,2022,Сыр полутвёрдый Эконива Щучанский Флоу-пак 200...,200 г,Классический,Эконива,800.631289,8167.40,0.680883,0.017582,2022-12


In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from scipy import stats
from scipy.special import inv_boxcox
from sklearn.metrics import mean_squared_log_error

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split



def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return adjusted_r2


# Установка стиля для графиков
sns.set(style="whitegrid")

# Параметры генерации данных
# START_DATE = datetime(2022, 1, 1)  # Начало периода
data=df.copy()

# 1. Обработка новых колонок для модели
# data['Promotion'] = data['SHARE_VOL_PROMO'].apply(lambda x: 1 if x > 0 else 0)  # Признак проведения промо
data['Package_Weight'] = data['Analytic1'].apply(lambda x: int(x.split()[0]))  # Преобразование веса в числовой формат
# data['Year_f'] = data['Date'].dt.year
data['Year'] = data['Date'].dt.year
data['Year_r'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Brand_r']=data['Brand'].values
data['Articul8_r']=data['Articul8'].values

# data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
# data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)

# # Скользящие средние
# for roll in [4]:
#   data[f'PRICE_PER_UNIT_roll_mean_{roll}'] = data['PRICE_PER_UNIT'].rolling(window=roll).mean()
#   data[f'SHARE_VOL_PROMO_roll_mean_{roll}'] = data['SHARE_VOL_PROMO'].rolling(window=roll).mean()

# # Создание лаговых признаков для целевой переменной SALES_VOL
# for lag in [1,2,3,6,24]:
#     data[f'SALES_VOL_lag_{lag}'] = data['SALES_VOL'].shift(lag)

# for lag in [1,2]:  # Лаги 1, 2, 3
#     df[f'PRICE_PER_UNIT_lag_{lag}'] = df['PRICE_PER_UNIT'].shift(lag)
#     df[f'SHARE_VOL_PROMO_lag_{lag}'] = df['SHARE_VOL_PROMO'].shift(lag)




# Список категориальных признаков
categorical_features = ['Brand', 'Articul8', 'Package_Weight']

# Создаем объект OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop=None)  # Используем sparse_output вместо sparse

# Применяем OneHotEncoder к категориальным признакам
encoded_features = encoder.fit_transform(data[categorical_features])


# Создаем DataFrame из закодированных признаков
encoded_df = pd.DataFrame(encoded_features.astype(int), columns=encoder.get_feature_names_out(categorical_features))

# Сброс индексов в обоих DataFrame, чтобы они совпадали
data.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Объединяем закодированные признаки с оригинальными данными
data = pd.concat([data.drop(columns=categorical_features), encoded_df], axis=1)
# data.drop(['Analytic1','Sku_name','SALES_VOL'],axis=1,inplace=True)


df_sales_vol_org = data[data['Year'] >= 2024].copy()



# Нормализация числовых признаков
numeric_cols = ['PRICE_PER_UNIT', 'SHARE_VOL_PROMO', 'Month', 'Year_r']
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])



# # Определение границ выбросов для целевой переменной
# Q1_y = data['SALES_VOL'].quantile(0.10)
# Q3_y = data['SALES_VOL'].quantile(0.90)
# IQR_y = Q3_y - Q1_y

# # Удаление выбросов в целевой переменной
# yt= (data['SALES_VOL'] >= (Q1_y - 1.5 * IQR_y)) & (data['SALES_VOL'] <= (Q3_y + 1.5 * IQR_y))
# data = data[yt]






# 4. Разделение данных на обучающую и тестовую выборки
# Используем данные до 2024 года в качестве обучающей выборки, 2024 год — тестовая выборка
train_df = data[data['Year'] < 2024]
test_df = data[data['Year'] >= 2024]

features = [i for i in data.columns.tolist() if i not in ['Analytic1','Sku_name','SALES_VOL','SHARE_VOL','Date','Month','Brand_r','Articul8_r','Year']] #'Month','Date','Year'
target = 'SALES_VOL'
X = data[features]
y = data[target]


X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]


# Конвертация данных в PyTorch тензоры
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


# Определим наш датасет
class SalesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Создаем DataLoader для тренировки и тестирования
train_dataset = SalesDataset(X_train_tensor, y_train_tensor)
test_dataset = SalesDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# # Определение модели
# class LinearRegressionModel(nn.Module):
#     def __init__(self, input_dim):
#         super(LinearRegressionModel, self).__init__()
#         self.linear = nn.Linear(input_dim, 1)  # Простая линейная регрессия

#     def forward(self, x):
#         return self.linear(x)
# Определение многослойной модели
class MultiLayerRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(MultiLayerRegressionModel, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 64)  # Первый скрытый слой (64 нейрона)
        self.hidden2 = nn.Linear(64, 32)         # Второй скрытый слой (32 нейрона)
        self.hidden3 = nn.Linear(32, 16)         # Третий скрытый слой (16 нейронов)
        self.output = nn.Linear(16, 1)           # Выходной слой
        self.relu = nn.ReLU()                    # Функция активации ReLU

    def forward(self, x):
        x = self.relu(self.hidden1(x))           # Применяем ReLU к первому слою
        x = self.relu(self.hidden2(x))           # Применяем ReLU ко второму слою
        x = self.relu(self.hidden3(x))           # Применяем ReLU к третьему слою
        x = self.output(x)                       # Выходной слой без активации
        return x

# Пользовательская функция потерь RMSE
def rmse_loss(predictions, targets):
    return torch.sqrt(nn.MSELoss()(predictions, targets))

# Инициализация модели, функции потерь и оптимизатора
# model = LinearRegressionModel(input_dim=X_train.shape[1])
model = MultiLayerRegressionModel(input_dim=X_train.shape[1])
# criterion = nn.MSELoss()  # Среднеквадратичная ошибка
criterion = rmse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Обучение модели
epochs = 100
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_loader:
        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Вычисление потерь
        loss = criterion(outputs, targets)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

    # Печать потерь на каждой эпохе
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Оценка модели
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)

    mae = mean_absolute_error(y_test_tensor, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_tensor, y_pred))
    r2 = r2_score(y_test_tensor, y_pred)
    adjusted_r2 = adjusted_r2_score(y_test_tensor, y_pred, len(y_test_tensor), y_test_tensor.shape[1])

    print(f"\nОценка модели на тестовой выборке:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")
    print(f"Скорректированное R²: {adjusted_r2:.4f}")



Epoch [10/100], Loss: 285.7110
Epoch [20/100], Loss: 6633.9932
Epoch [30/100], Loss: 1202.0312
Epoch [40/100], Loss: 3602.5034
Epoch [50/100], Loss: 133.7961
Epoch [60/100], Loss: 5204.7534
Epoch [70/100], Loss: 5751.9473
Epoch [80/100], Loss: 811.7289
Epoch [90/100], Loss: 1908.3641
Epoch [100/100], Loss: 9265.1318

Оценка модели на тестовой выборке:
MAE: 18539.99
RMSE: 24678.72
R²: -0.8105
Скорректированное R²: -0.8151


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from scipy import stats
from scipy.special import inv_boxcox
from sklearn.metrics import mean_squared_log_error

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split



def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return adjusted_r2


# Установка стиля для графиков
sns.set(style="whitegrid")

# Параметры генерации данных
# START_DATE = datetime(2022, 1, 1)  # Начало периода
data=df.copy()

# 1. Обработка новых колонок для модели
# data['Promotion'] = data['SHARE_VOL_PROMO'].apply(lambda x: 1 if x > 0 else 0)  # Признак проведения промо
data['Package_Weight'] = data['Analytic1'].apply(lambda x: int(x.split()[0]))  # Преобразование веса в числовой формат
# data['Year_f'] = data['Date'].dt.year
data['Year'] = data['Date'].dt.year
data['Year_r'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Brand_r']=data['Brand'].values
data['Articul8_r']=data['Articul8'].values

# data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
# data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)

# # Скользящие средние
# for roll in [4]:
#   data[f'PRICE_PER_UNIT_roll_mean_{roll}'] = data['PRICE_PER_UNIT'].rolling(window=roll).mean()
#   data[f'SHARE_VOL_PROMO_roll_mean_{roll}'] = data['SHARE_VOL_PROMO'].rolling(window=roll).mean()

# # Создание лаговых признаков для целевой переменной SALES_VOL
# for lag in [1,2,3,6,24]:
#     data[f'SALES_VOL_lag_{lag}'] = data['SALES_VOL'].shift(lag)

# for lag in [1,2]:  # Лаги 1, 2, 3
#     df[f'PRICE_PER_UNIT_lag_{lag}'] = df['PRICE_PER_UNIT'].shift(lag)
#     df[f'SHARE_VOL_PROMO_lag_{lag}'] = df['SHARE_VOL_PROMO'].shift(lag)




# Список категориальных признаков
categorical_features = ['Brand', 'Articul8', 'Package_Weight']

# Создаем объект OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop=None)  # Используем sparse_output вместо sparse

# Применяем OneHotEncoder к категориальным признакам
encoded_features = encoder.fit_transform(data[categorical_features])


# Создаем DataFrame из закодированных признаков
encoded_df = pd.DataFrame(encoded_features.astype(int), columns=encoder.get_feature_names_out(categorical_features))

# Сброс индексов в обоих DataFrame, чтобы они совпадали
data.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Объединяем закодированные признаки с оригинальными данными
data = pd.concat([data.drop(columns=categorical_features), encoded_df], axis=1)
# data.drop(['Analytic1','Sku_name','SALES_VOL'],axis=1,inplace=True)


df_sales_vol_org = data[data['Year'] >= 2024].copy()



# Нормализация числовых признаков
numeric_cols = ['PRICE_PER_UNIT', 'SHARE_VOL_PROMO', 'Month', 'Year_r']
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])



# # Определение границ выбросов для целевой переменной
# Q1_y = data['SALES_VOL'].quantile(0.10)
# Q3_y = data['SALES_VOL'].quantile(0.90)
# IQR_y = Q3_y - Q1_y

# # Удаление выбросов в целевой переменной
# yt= (data['SALES_VOL'] >= (Q1_y - 1.5 * IQR_y)) & (data['SALES_VOL'] <= (Q3_y + 1.5 * IQR_y))
# data = data[yt]






# 4. Разделение данных на обучающую и тестовую выборки
# Используем данные до 2024 года в качестве обучающей выборки, 2024 год — тестовая выборка
train_df = data[data['Year'] < 2024]
test_df = data[data['Year'] >= 2024]

features = [i for i in data.columns.tolist() if i not in ['Analytic1','Sku_name','SALES_VOL','SHARE_VOL','Date','Month','Brand_r','Articul8_r','Year']] #'Month','Date','Year'
target = 'SALES_VOL'
X = data[features]
y = data[target]


X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]


# Конвертация данных в PyTorch тензоры
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




# Функция для расчета скорректированного R²
def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Изменим форму данных для LSTM (batch_size, sequence_length, num_features)
# Временные ряды предполагают использование последовательности данных, для этого разбиваем данные на последовательности
def create_sequences(X, y, seq_length=3):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

# Создание последовательностей для LSTM
seq_length = 4  # Длина последовательности, которую учитывает LSTM
X_train_seq, y_train_seq = create_sequences(X_train_tensor, y_train_tensor, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test_tensor, y_test_tensor, seq_length)


# Конвертация в тензоры
X_train_seq_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_seq_tensor = torch.tensor(y_train_seq, dtype=torch.float32)
X_test_seq_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_test_seq_tensor = torch.tensor(y_test_seq, dtype=torch.float32)

# Определим наш датасет
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Создаем DataLoader для тренировки и тестирования
train_dataset = TimeSeriesDataset(X_train_seq_tensor, y_train_seq_tensor)
test_dataset = TimeSeriesDataset(X_test_seq_tensor, y_test_seq_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Определение модели на основе LSTM
class LSTMRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMRegressionModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Определение LSTM слоя
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Определение линейного слоя для предсказания
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Инициализация начального скрытого и клеточного состояния
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Прямой проход через LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Выбор последнего временного шага
        out = self.fc(out[:, -1, :])
        return out

# Пользовательская функция потерь RMSE
def rmse_loss(predictions, targets):
    return torch.sqrt(nn.MSELoss()(predictions, targets))

# Инициализация модели, функции потерь и оптимизатора
input_dim = X_train_seq_tensor.shape[2]  # Количество признаков
hidden_dim = 64                         # Количество нейронов в LSTM
num_layers = 2                          # Количество слоев LSTM
output_dim = 1                          # Одно выходное значение

model = LSTMRegressionModel(input_dim, hidden_dim, num_layers, output_dim)
criterion = rmse_loss  # Задаем RMSE как функцию потерь
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
# Инициализация оптимизатора SGD
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # Используем SGD с моментом 0.9


# Списки для сохранения значений потерь на каждой эпохе
train_losses = []
val_losses = []
# Обучение модели
epochs = 1000
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Вычисление потерь
        loss = criterion(outputs, targets)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

         # Накопление потерь для вывода
        train_loss += loss.item()

    # Рассчитываем среднюю потерю за эпоху
    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # Оценка на тестовом наборе
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()


    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    # Печать потерь на каждой эпохе
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train RMSE: {train_loss:.4f}, Validation RMSE: {val_loss:.4f}')



# Построение графиков потерь
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label='Train RMSE Loss')
plt.plot(val_losses, label='Validation RMSE Loss')
plt.xlabel('Epochs')
plt.ylabel('RMSE Loss')
plt.title('Train and Validation RMSE Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# Оценка модели на тестовой выборке с расчетом дополнительных метрик
with torch.no_grad():
    y_pred = model(X_test_seq_tensor)

    # Преобразуем тензоры в numpy массивы
    y_pred_np = y_pred.numpy()
    y_test_np = y_test_seq_tensor.numpy()

    # Расчет метрик
    mae = mean_absolute_error(y_test_np, y_pred_np)
    rmse = np.sqrt(mean_squared_error(y_test_np, y_pred_np))
    r2 = r2_score(y_test_np, y_pred_np)
    adjusted_r2 = adjusted_r2_score(y_test_np, y_pred_np, len(y_test_np), X_test_seq_tensor.shape[2])

    # Печать метрик
    print(f"\nОценка модели на тестовой выборке:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")
    print(f"Скорректированное R²: {adjusted_r2:.4f}")





Epoch [10/1000], Train RMSE: 16180.7448, Validation RMSE: 13037.4663
Epoch [20/1000], Train RMSE: 15871.1771, Validation RMSE: 12868.8265
Epoch [30/1000], Train RMSE: 15598.8661, Validation RMSE: 12836.4243
Epoch [40/1000], Train RMSE: 15899.0212, Validation RMSE: 12852.6939
Epoch [50/1000], Train RMSE: 15636.0197, Validation RMSE: 12892.0559
Epoch [60/1000], Train RMSE: 15140.3131, Validation RMSE: 12055.1246
Epoch [70/1000], Train RMSE: 14600.5622, Validation RMSE: 11747.5145
Epoch [80/1000], Train RMSE: 14204.1916, Validation RMSE: 11814.9477
Epoch [90/1000], Train RMSE: 14262.5748, Validation RMSE: 11722.3175
Epoch [100/1000], Train RMSE: 13925.1350, Validation RMSE: 11251.4063
Epoch [110/1000], Train RMSE: 13751.8605, Validation RMSE: 12004.3235
Epoch [120/1000], Train RMSE: 13544.6451, Validation RMSE: 11464.4548
Epoch [130/1000], Train RMSE: 13466.0294, Validation RMSE: 11668.3349
Epoch [140/1000], Train RMSE: 13599.8043, Validation RMSE: 11731.1285
Epoch [150/1000], Train RMSE:

In [40]:
X_train_seq.shape

(1632, 4, 118)

In [41]:
y_train_seq.shape

(1632, 1)

In [ ]:

mtn ="RMSE"


mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
adjusted_r2 = adjusted_r2_score(y_test, y_pred, len(y_test), X_test.shape[1])

print(f"\nОценка модели на тестовой выборке:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"Скорректированное R²: {adjusted_r2:.4f}")
# rmse_original = np.expm1(rmse)  # Обратное преобразование RMSE
# print(f"Обратное RMSE: {rmse_original}")
# print(f"Mean Squared Log Error: {mean_squared_log_error(abs(y_test), abs(y_pred))}")



# Получаем значения ошибок на каждой итерации
train_errors = model.get_evals_result()['learn'][mtn]
test_errors = model.get_evals_result()['validation'][mtn]

# Построение графика
plt.figure(figsize=(10, 6))
plt.plot(train_errors, label='Ошибка на обучающих данных')
plt.plot(test_errors, label='Ошибка на тестовых данных')
plt.xlabel('Итерации')
plt.ylabel('RMSE')
plt.title('Процесс обучения модели CatBoost')
plt.legend()
plt.grid()
plt.show()

# 7. Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Фактические vs Прогнозные продажи')
plt.xlabel('Фактические продажи')
plt.ylabel('Прогнозные продажи')
plt.tight_layout()
plt.show()

# Распределение ошибок
errors = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(errors, bins=20, kde=True, color='salmon',hue=None)
plt.title('Распределение ошибок (Фактические - Прогнозные)')
plt.xlabel('Ошибка')
plt.ylabel('Частота')
plt.tight_layout()
plt.show()

# Важность признаков
feature_importances = model.get_feature_importance()
features_names = features
feature_importance_df = pd.DataFrame(data=model.get_feature_importance(),index=features)

feature_importance_df = feature_importance_df.sort_values(by=0,ascending=False).head(20)

plt.figure(figsize=(10,6))
sns.barplot(x=feature_importance_df[0], y=feature_importance_df.index)
plt.title('Важность признаков')
plt.xlabel('Важность')
plt.ylabel('Признаки')
plt.tight_layout()
plt.show()
#________________________________________
# 8. Кросс-валидация
# kf = KFold(n_splits=5, shuffle=True, random_state=42,)
# cv_mae = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')
# cv_rmse = cross_val_score(model, X, y, cv=kf, scoring='neg_root_mean_squared_error')
# cv_r2 = cross_val_score(model, X, y, cv=kf, scoring='r2')

# print("\nКросс-валидация (5 фолдов):")
# print(f"Средний MAE: {-cv_mae.mean():.2f} ± {cv_mae.std():.2f}")
# print(f"Средний RMSE: {-cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}")
# print(f"Средний R²: {cv_r2.mean():.4f} ± {cv_r2.std():.4f}")

# 9. Интеграция прогнозов в тестовую выборку
test_df = test_df.copy()
# test_df['SALES_VOL']=df_sales_vol_org['SALES_VOL'].values

# Добавляем прогнозные значения в новую колонку
test_df['Predicted_Sales'] = y_pred

# Преобразование 'Date' в формат datetime перед группировкой
test_df['Date'] = pd.to_datetime(test_df['Date'].dt.to_timestamp())


# 10. Построение линейных графиков динамики продаж
def plot_comparative_line(data, x, y_actual, y_predicted, title, xlabel, ylabel, rotation=45):
    plt.figure(figsize=(12, 6))
    plt.plot(data[x], data[y_actual], marker='o', label='Фактические продажи')
    plt.plot(data[x], data[y_predicted], marker='x', linestyle='--', label='Прогнозные продажи')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=rotation)
    plt.legend()
    plt.ylim(0, None)
    plt.tight_layout()
    plt.show()

# Сравнение фактических и прогнозных продаж по месяцам
monthly_sales = test_df.groupby('Date').agg({'SALES_VOL': 'sum', 'Predicted_Sales': 'sum'}).reset_index()
plot_comparative_line(
    monthly_sales,
    x='Date',
    y_actual='SALES_VOL',
    y_predicted='Predicted_Sales',
    title='Динамика суммарных продаж по месяцам: Фактические vs Прогнозные',
    xlabel='Месяц',
    ylabel='Суммарные продажи'
)

# 11. Сравнение суммарных продаж по Brand (фактические и прогнозные)
# Декодирование категориальных признаков обратно в исходные значения
# def decode_labels(df, column, label_encoder):
#     inverse_mapping = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))
#     return df[column].map(inverse_mapping)

# test_df['Brand'] = decode_labels(test_df, 'Brand_r', data['Brand_r'])
# test_df['Articul8'] = decode_labels(test_df, 'Articul8_r', data['Articul8_r'])

# Суммарные продажи по брендам
sales_per_brand_actual = test_df.groupby('Brand_r')['SALES_VOL'].sum().reset_index()
sales_per_brand_predicted = test_df.groupby('Brand_r')['Predicted_Sales'].sum().reset_index()
sales_per_brand = pd.merge(sales_per_brand_actual, sales_per_brand_predicted, on='Brand_r', how='left')
sales_per_brand.rename(columns={'SALES_VOL': 'Sales_Actual', 'Predicted_Sales': 'Sales_Predicted'}, inplace=True)

print("\nСуммарные фактические и прогнозные продажи по Brand:")
print(sales_per_brand.head())

# 12. Линейный график динамики продаж по брендам
top_n_brand = 5
top_brands = sales_per_brand.sort_values('Sales_Actual', ascending=False).head(top_n_brand)['Brand_r'].tolist()

for brand in top_brands:
    brand_data = test_df[test_df['Brand_r'] == brand]
    monthly_brand_actual = brand_data.groupby('Date')['SALES_VOL'].sum().reset_index()
    monthly_brand_predicted = brand_data.groupby('Date')['Predicted_Sales'].sum().reset_index()
    monthly_brand = pd.merge(monthly_brand_actual, monthly_brand_predicted, on='Date', how='left')
    monthly_brand.rename(columns={'SALES_VOL': 'Sales_Actual', 'Predicted_Sales': 'Sales_Predicted'}, inplace=True)
    plot_comparative_line(
        monthly_brand,
        x='Date',
        y_actual='Sales_Actual',
        y_predicted='Sales_Predicted',
        title=f'Динамика продаж для {brand}: Фактические vs Прогнозные',
        xlabel='Месяц',
        ylabel='Суммарные продажи',
        rotation=45
    )

# 13. Сравнение суммарных продаж по типам продукта (фактические и прогнозные)
sales_per_articul8_actual = test_df.groupby('Articul8_r')['SALES_VOL'].sum().reset_index()
sales_per_articul8_predicted = test_df.groupby('Articul8_r')['Predicted_Sales'].sum().reset_index()
sales_per_articul8 = pd.merge(sales_per_articul8_actual, sales_per_articul8_predicted, on='Articul8_r', how='left')
sales_per_articul8.rename(columns={'SALES_VOL': 'Sales_Actual', 'Predicted_Sales': 'Sales_Predicted'}, inplace=True)

print("\nСуммарные фактические и прогнозные продажи по типам продукта (Articul8_r):")
print(sales_per_articul8.head())

# 14. Линейный график динамики продаж по типам продукта
plot_comparative_line(
    sales_per_articul8,
    x='Articul8_r',
    y_actual='Sales_Actual',
    y_predicted='Sales_Predicted',
    title='Динамика суммарных продаж по типам продукта: Фактические vs Прогнозные',
    xlabel='Тип продукта',
    ylabel='Суммарные продажи',
    rotation=90
)

# 15. Линейный график динамики продаж по месяцам для каждого типа продукта
plt.figure(figsize=(14, 8))
articul8_types_unique = sales_per_articul8['Articul8_r'].unique()
for articul8 in articul8_types_unique:
    articul8_data = test_df[test_df['Articul8_r'] == articul8]
    monthly_articul8_actual = articul8_data.groupby('Date')['SALES_VOL'].sum().reset_index()
    monthly_articul8_predicted = articul8_data.groupby('Date')['Predicted_Sales'].sum().reset_index()
    monthly_articul8 = pd.merge(monthly_articul8_actual, monthly_articul8_predicted, on='Date', how='left')
    monthly_articul8.rename(columns={'SALES_VOL': 'Sales_Actual', 'Predicted_Sales': 'Sales_Predicted'}, inplace=True)
    plt.plot(monthly_articul8['Date'], monthly_articul8['Sales_Actual'], marker='o', label=f'{articul8} Фактические')
    plt.plot(monthly_articul8['Date'], monthly_articul8['Sales_Predicted'], marker='x', linestyle='--', label=f'{articul8} Прогнозные')

plt.title('Динамика суммарных продаж по типам продукта по месяцам: Фактические vs Прогнозные')
plt.xlabel('Месяц')
plt.ylabel('Суммарные продажи')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 16. Линейный график динамики продаж по брендам для каждого Brand
plt.figure(figsize=(14, 8))
for brand in top_brands:
    brand_data = test_df[test_df['Brand_r'] == brand]
    monthly_brand_actual = brand_data.groupby('Date')['SALES_VOL'].sum().reset_index()
    monthly_brand_predicted = brand_data.groupby('Date')['Predicted_Sales'].sum().reset_index()
    monthly_brand = pd.merge(monthly_brand_actual, monthly_brand_predicted, on='Date', how='left')
    monthly_brand.rename(columns={'SALES_VOL': 'Sales_Actual', 'Predicted_Sales': 'Sales_Predicted'}, inplace=True)
    plt.plot(monthly_brand['Date'], monthly_brand['Sales_Actual'], marker='o', label=f'{brand} Фактические')
    plt.plot(monthly_brand['Date'], monthly_brand['Sales_Predicted'], marker='x', linestyle='--', label=f'{brand} Прогнозные')

plt.title(f'Динамика суммарных продаж по топ-{top_n_brand} Brand_r по месяцам: Фактические vs Прогнозные')
plt.xlabel('Месяц')
plt.ylabel('Суммарные продажи')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

print("\nПостроение линейных графиков завершено успешно.")

